sqlalchemy.org - tutorial

# Establishing database connection

* The Engine  - global object created once for particular database server
* The URL string - how to connect (see connectionstrings.com)

In [12]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL #method to transform connection string into url string
from urllib.parse import unquote

# create connection string in correct format
server_name   = "localhost"
database_name = "everyloop"

connection_string = f"DRIVER=ODBC Driver 17 for SQL Server;SERVER={server_name};DATABASE={database_name};Trusted_Connection=yes"
url_string        = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

print('Connecting to database using URL string:')
unquoted_url = unquote(str(url_string))
print(unquoted_url, '\n')


try:    
    engine = create_engine(url_string) # create engine object
    with engine.connect() as connection: # NOTE with-statement (Python manager form) connection object (is not used here)
        print(f'Successfully connected to {database_name}!')
except Exception as e:
    print('Error while connecting to database:\n')
    print(e)

Connecting to database using URL string:
mssql+pyodbc://?odbc_connect=DRIVER=ODBC+Driver+17+for+SQL+Server;SERVER=localhost;DATABASE=everyloop;Trusted_Connection=yes 

Successfully connected to everyloop!


# Query database using raw SQL

text() - Simplest, old

In [14]:
from sqlalchemy import text

query = text("SELECT TOP 5 Id, FirstName, LastName, Email FROM USERS ORDER BY FirstName, LastName") # NOTE text()
type(query)

sqlalchemy.sql.elements.TextClause

## The Result object
connection.execute() SQLAlchemy submits query to te server which responds by sending beack the requested data as Result Object by connecttion.execute(). Best practice to consume this object witing the "connect" block. Below examples on how:

### Execute query and fetch all rows from result

In [15]:
with engine.connect() as conn:
    result = conn.execute(query) # syncron programming (code stops here waiting for responce from server and continues once resp is recieved)

    print(result.fetchall()) # result is a Resul object which has a number of funcions such as fetchall()

# asyncio library for async/await syntax (asyncron programmig)

[('741109-2058', 'Alexander', 'Dahl', 'alexander.dahl@telia.se'), ('530720-7675', 'Alvin', 'Lindholm', 'alvin.lindholm@gmail.com'), ('820624-3075', 'Anders', 'Hansson', 'anders.hansson@hotmail.com'), ('751123-9724', 'Anne', 'Åkerman', 'anne.akerman@hotmail.com'), ('620925-4245', 'Annette', 'Bergfalk', 'annette.bergfalk@telia.se')]


In [16]:
print(type(result))
print(type(result.fetchall()))

<class 'sqlalchemy.engine.cursor.CursorResult'>
<class 'list'>


### Execute query and iterate through result

In [7]:
with engine.connect() as conn:
    result = conn.execute(query)

    for row in result:
        print(f"{row.FirstName} {row.LastName}")

Alexander Dahl
Alvin Lindholm
Anders Hansson
Anne Åkerman
Annette Bergfalk


### Iterate column names and fields

In [8]:
with engine.connect() as conn:
    result = conn.execute(query)

    for col_name in result.keys():
        print(col_name.upper().ljust(20), end='')
    print()

    for row in result:
        for field in row:
            print(field.ljust(20), end='')
        print()

ID                  FIRSTNAME           LASTNAME            EMAIL               
741109-2058         Alexander           Dahl                alexander.dahl@telia.se
530720-7675         Alvin               Lindholm            alvin.lindholm@gmail.com
820624-3075         Anders              Hansson             anders.hansson@hotmail.com
751123-9724         Anne                Åkerman             anne.akerman@hotmail.com
620925-4245         Annette             Bergfalk            annette.bergfalk@telia.se


### Create DataFrames from query result

In [17]:
import pandas as pd

df = pd.read_sql_query(query, con=engine, index_col='Id')
df

,FirstName,LastName,Email
Id,,,
741109-2058,Alexander,Dahl,alexander.dahl@telia.se
530720-7675,Alvin,Lindholm,alvin.lindholm@gmail.com
820624-3075,Anders,Hansson,anders.hansson@hotmail.com
751123-9724,Anne,Åkerman,anne.akerman@hotmail.com
620925-4245,Annette,Bergfalk,annette.bergfalk@telia.se


: 

### Example search application (unsafe implmentation) - SQL injection
Type commands via input window to make changes in databases. For example:
* ';--'                                                                 get all data
* ';drop table users3; --'                                              drop data
* ' and 1=0 union select firstname, lastname, password from users; --'  get data (and 1 = 0 makes sure will get empty result; union own query; -- comments away rest)
* ' and 1=0 union select 1, 2, 3; --'                                   write in own data in database

To avoid:
* limit rights for users who log on
* never allow user input in any part of our queries so no concatination in queries
    * query = "select * from users where username = '" + username_input + "';"
* user data in queries should always be parameterized so recieve SQL statment and user data separately (static statement) 
    * query = "select * from users where username = @username_input"
* 

#### Bound parameters in SQLAlchemy - parametirized query
text() allows to bind parameters using the named colon format (:param).
Bound parameter *:first_name* in the query and the pass in the value of the python varable *name* when eqecuting query.
* *T-SQL uses @param, while SQLAlchemy uses :param*

In [10]:
name = "';--"

parameterized_query = text("Select * from Users where FirstName like '%' + :first_name + '%'")  # hard coded query indicating there is a parameter :first_name to be expected

with engine.connect() as conn:
    result = conn.execute(parameterized_query, {"first_name": name}) # defining what :first_name is
    print(result.fetchall())

[]


In [11]:
search_string = input("Search airports: ")

print(f"Search string: {search_string}\n")

# query as f-string
unsafe_query = f"""
select top 10
    IATA, 
    [Airport name] as 'name', 
    [Location served] as 'location' 
from 
    airports 
where 
    [Location served] like '%{search_string}%'"""

# sending request and getting Result object
with engine.connect() as conn:    
    result = conn.execute(text(unsafe_query)) # NOTE text()

    print(f"{'IATA'.ljust(8)}{'Airport name'.ljust(50)}{'Location'}")
    
    for airport in result:
        print(f"{str(airport.IATA).ljust(8)}{str(airport.name).ljust(50)}{airport.location}")

    print(unsafe_query)

Search string: anna

IATA    Airport name                                      Location
AAE     Rabah Bitat Airport (Les Salines Airport)         Annaba, Algeria
ANP     Lee Airport                                       Annapolis, Maryland, United States
AOB     Annanberg Airport                                 Annanberg, Papua New Guinea
CJF     Coondewanna Airport                               Coondewanna, Western Australia, Australia
CNN     Kannur International Airport                      kannur, (Cannanore) Kerala, India
DNP     Tribhuvannagar Airport (Dang Airport)             Tribhuwannagar, Nepal
MNH     Al-Musannah Airport[1]                            Al-Musannah, Oman
NAI     Annai Airport                                     Annai, Guyana
SAV     Savannah/Hilton Head International Airport        Savannah, Georgia, United States
SVN     Hunter Army Airfield                              Savannah, Georgia, United States

select top 10
    IATA, 
    [Airport name] as 'name', 
